<a href="https://colab.research.google.com/github/Guptaraj06/optimal-Fertilizer/blob/main/Optimal_Fertilizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

playground_series_s5e6_path = kagglehub.competition_download('playground-series-s5e6')
irakozekelly_fertilizer_prediction_path = kagglehub.dataset_download('irakozekelly/fertilizer-prediction')

print('Data source import complete.')


In [ ]:
import pandas as pd

In [ ]:
train_df=pd.read_csv('/kaggle/input/playground-series-s5e6/train.csv',index_col='id')

In [ ]:
train_df

In [ ]:
train_df.columns.tolist()

In [ ]:
train_df['Fertilizer Name'].value_counts()

In [ ]:
a=set(x for i in train_df['Fertilizer Name'].apply(lambda x:list(x.split(" "))) for x in i)

In [ ]:
a

In [ ]:
from lightgbm import LGBMClassifier,log_evaluation,early_stopping
from sklearn.feature_selection import mutual_info_classif
import numpy as np
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.base import clone
import matplotlib.pyplot as plt
import gc

In [ ]:
test_df=pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv',index_col='id')
O_df=pd.read_csv('/kaggle/input/fertilizer-prediction/Fertilizer Prediction.csv')

In [ ]:
O_df.info()

In [ ]:
target='Fertilizer Name'
n_folds=5
seed=42

In [ ]:
train_df.columns

In [ ]:
features=test_df.columns.tolist()
train_df[features]=train_df[features].astype('str').astype('category')
test_df[features]=test_df[features].astype('str').astype('category')
O_df[features]=O_df[features].astype('str').astype('category')

le=LabelEncoder()
train_df[target]=le.fit_transform(train_df[target])
O_df[target]=le.transform(O_df[target])

X=train_df.drop(['Fertilizer Name'],axis=1)
y=train_df[target]
X_orig=O_df.drop([target],axis=1)
y_orig=O_df[target]
X_test=test_df

In [ ]:
X_info=mutual_info_classif(X,y,random_state=seed,discrete_features=True)
X_info=pd.Series(X_info)
X_info.index=X.columns
X_info=pd.DataFrame(X_info.sort_values(ascending=False),columns=['Mutual Information'])
X_info.style.bar(subset=['Mutual Information'],cmap="RdYlGn")

In [ ]:
X_info=mutual_info_classif(X_orig,y_orig,random_state=seed,discrete_features=True)
X_info=pd.Series(X_info)
X_info.index=X.columns
X_info=pd.DataFrame(X_info.sort_values(ascending=False),columns=['Mutual Information'])
X_info.style.bar(subset=['Mutual Information'],cmap="RdYlGn")

In [ ]:
def map3(y_true, y_pred_probs):
    y_true = [[x] for x in y_true]
    y_pred_probs = np.argsort(y_pred_probs, axis=1)[:, -3:][:, ::-1].tolist()

    def ap3(y_true, y_pred_probs):
        y_pred_probs = y_pred_probs[:3]

        score = 0.0
        num_hits = 0.0

        for i,p in enumerate(y_pred_probs):
            if p in y_true and p not in y_pred_probs[:i]:
                num_hits += 1.0
                score += num_hits / (i+1.0)

        if not y_true:
            return 0.0

        return score

    return np.mean([ap3(a,p) for a,p in zip(y_true, y_pred_probs)])

In [ ]:
class Trainer:
    def __init__(self, model):
        self.model = model


    def fit_predict(self, X, y, X_test, X_original=None, y_original=None, fit_args={}):
        print(f"Training {self.model.__class__.__name__}\n")

        scores = []
        oof_pred_probs = np.zeros((X.shape[0], 7))
        test_pred_probs = np.zeros((X_test.shape[0], 7))

        skf = StratifiedKFold(n_splits=n_folds, random_state=seed, shuffle=True)
        for fold_idx, (train_idx, val_idx) in enumerate(skf.split(X, y)):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y[train_idx], y[val_idx]

            if X_original is not None and y_original is not None:
                X_train = pd.concat([X_train, X_original])
                y_train = pd.concat([y_train, y_original])

            model = clone(self.model)

            if fit_args:
                model.fit(X_train, y_train, **fit_args, eval_set=[(X_val, y_val)])
            else:
                model.fit(X_train, y_train)

            y_pred_probs = model.predict_proba(X_val)
            oof_pred_probs[val_idx] = y_pred_probs

            temp_test_pred_probs = model.predict_proba(X_test)
            test_pred_probs += temp_test_pred_probs / n_folds

            score = map3(y_val, y_pred_probs)
            scores.append(score)

            del model, X_train, y_train, X_val, y_val, y_pred_probs
            gc.collect()

            if fit_args:
                print(f"\n--- Fold {fold_idx + 1} - MAP@3: {score:.6f}\n\n")
            else:
                print(f"--- Fold {fold_idx + 1} - MAP@3: {score:.6f}")

        overall_score = map3(y, oof_pred_probs)

        print(f"\n------ Overall MAP@3: {overall_score:.6f} | Average MAP@3: {np.mean(scores):.6f} ± {np.std(scores):.6f}")

        return oof_pred_probs, test_pred_probs, scores

    def tune(self, X, y):
        scores = []

        skf = StratifiedKFold(n_splits=n_folds, random_state=seed, shuffle=True)
        for fold_idx, (train_idx, val_idx) in enumerate(skf.split(X, y)):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y[train_idx], y[val_idx]

            model = clone(self.model)
            model.fit(X_train, y_train)

            y_pred_probs = model.predict_proba(X_val)
            score = map3(y_val, y_pred_probs)
            scores.append(score)

            del model, X_train, y_train, X_val, y_val, y_pred_probs
            gc.collect()

        return np.mean(scores)

In [ ]:
scores={}
oof_pred_probs={}
test_pred_probs={}

In [ ]:
lgbm_params={
  "boosting_type":"gbdt",
  "device":"gpu",
  "colsample_bytree":0.4366677273946288,
  "learning_rate":0.016164161953515117,
  "max_depth":12,
  "min_child_samples":67,
  "n_estimators":10000,
  "n_jobs":-1,
  "num_leaves":243,
  "random_state":42,
  "reg_alpha":6.38288560443373,
  "reg_lambda":9.392999314379155,
  "subsample":0.7989164499431718,
  "verbose":-1
}

lgbm_goss_params={
    "boosting_type":"goss",
    "device":"gpu",
    "colsample_bytree":0.32751831793031183,
    "learning_rate":0.006700715059604966,
    "max_depth":12,
    "min_child_samples":84,
    "n_estimators":10000,
    "n_jobs":-1,
    "num_leaves":229,
    "random_state":42,
    "reg_alpha":6.879977008084246,
    "reg_lambda":4.739518466581721,
    "subsample":0.5411572049978781,
    "verbose":-1
}

xgb_params={
   "device":"gpu",
   "max_depth":12,
   "colsample_bytree":0.467,
   "subsample":0.86,
   "n_estimators":10000,
   "learning_rate":0.03,
   "gamma":0.26,
   "max_delta_step":4,
   "reg_alpha":2.7,
   "reg_lambda":1.4,
   "early_stopping_rounds":100,
   "objective":'multi:softprob',
   "random_state":13,
   "enable_categorical":True
}

In [ ]:
lgbm_model = LGBMClassifier(**lgbm_params)
lgbm_trainer = Trainer(lgbm_model)

fit_args = {
    "callbacks": [
        log_evaluation(period=1000),
        early_stopping(stopping_rounds=100)
    ]
}

oof_pred_probs["LightGBM (gbdt)"], test_pred_probs["LightGBM (gbdt)"], scores["LightGBM (gbdt)"] = lgbm_trainer.fit_predict(X, y, X_test, X_orig, y_orig, fit_args)

In [ ]:
print(oof_pred_probs)

In [ ]:
lgbm_goss_model=LGBMClassifier(**lgbm_goss_params)
lgbm_trainer=Trainer(lgbm_goss_model)
fit_args={
    "callbacks":[
        early_stopping(stopping_rounds=100),
        log_evaluation(period=1000)
    ]
}
oof_pred_probs['LightGBM(goss)'],test_pred_probs['LightGBM(goss)'],scores['LightGBM(goss)']=lgbm_trainer.fit_predict(X,y,X_test,X_orig,y_orig,fit_args)

In [ ]:
xgb_model = XGBClassifier(**xgb_params)
xgb_trainer = Trainer(xgb_model)

fit_args = {
    "verbose": 1000
}

oof_pred_probs["XGBoost"], test_pred_probs["XGBoost"], scores["XGBoost"] = xgb_trainer.fit_predict(X, y, X_test, X_orig, y_orig, fit_args)

In [ ]:
import pickle
import joblib

with open('/kaggle/working/result.pkl','wb') as f:
    pickle.dump({'c1':oof_pred_probs,'c2':test_pred_probs,'c3':scores},f)

In [ ]:
X=pd.DataFrame(np.concatenate(list(oof_pred_probs.values()),axis=1))
X_test=pd.DataFrame(np.concatenate(list(test_pred_probs.values()),axis=1))

In [ ]:
lr_model=LogisticRegression(random_state=42,max_iter=10000,solver='liblinear',penalty='l2',
                            C=32.89802104596641,
                            tol=0.0029878837974181643,
                            fit_intercept=True)
lr_trainer=Trainer(lr_model)
_,lr_test_pred_probs,scores['Ensemble']=lr_trainer.fit_predict(X,y,X_test)

In [ ]:
final_predictions=[]
for i in np.argsort(lr_test_pred_probs)[:,-3:][:,::-1]:
  prediction=le.inverse_transform(i)
  final_predictions.append(" ".join(prediction))

In [ ]:
sub=pd.read_csv('/kaggle/input/playground-series-s5e6/sample_submission.csv')
sub[target]=final_predictions
sub.to_csv(f'sub_logistic_regression_{np.mean(scores['Ensemble']):.6f}.csv',index=False)

In [ ]:
scores = pd.DataFrame(scores)
mean_scores = scores.mean().sort_values(ascending=False)
order = scores.mean().sort_values(ascending=False).index.tolist()

min_score = mean_scores.min()
max_score = mean_scores.max()

padding = (max_score - min_score) * 0.5
lower_limit = min_score - padding
upper_limit = max_score + padding

fig, axs = plt.subplots(1, 2, figsize=(15, scores.shape[1] * 0.4))

boxplot = sns.boxplot(data=scores, order=order, ax=axs[0], orient="h", color="grey")
axs[0].set_title(f"Fold MAP@3")
axs[0].set_xlabel("")
axs[0].set_ylabel("")

barplot = sns.barplot(x=mean_scores.values, y=mean_scores.index, ax=axs[1], color="grey")
axs[1].set_title(f"Mean MAP@3")
axs[1].set_xlabel("")
axs[1].set_xlim(left=lower_limit, right=upper_limit)
axs[1].set_ylabel("")

for i, (score, model) in enumerate(zip(mean_scores.values, mean_scores.index)):
    color = "cyan" if "ensemble" in model.lower() else "grey"
    barplot.patches[i].set_facecolor(color)
    boxplot.patches[i].set_facecolor(color)
    barplot.text(score, i, round(score, 6), va="center")

plt.tight_layout()
plt.show()